# AI Applications Project: Report Generation

In [ ]:
#Importing the generative ai
%pip install -U -q "google-genai>=1.4.0"

Note: you may need to restart the kernel to use updated packages.


### Setting up API Key

In [ ]:
from google import genai

GOOGLE_API_KEY = input("Enter your Google API key:")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [84]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro","gemini-2.0-flash"] {"allow-input":true, isTemplate: true}

# Data

## Database Integration

In [91]:
import numpy as np
import pandas as pd
import os
import sys

sys.path.append('../../')

from Database.db import SessionLocal
from Database_Table import Inventory, Order

def getDbContent():
    session = SessionLocal()
    inventory_records = session.query(Inventory).all()
    order_records = session.query(Order).all()
    session.close()
    return inventory_records, order_records

inventory, order = getDbContent()

2025-08-17 19:06:09,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-17 19:06:09,698 INFO sqlalchemy.engine.Engine SELECT `Inventory`.`ItemId` AS `Inventory_ItemId`, `Inventory`.`ItemName` AS `Inventory_ItemName`, `Inventory`.`ItemCategory` AS `Inventory_ItemCategory`, `Inventory`.`ItemQuantity` AS `Inventory_ItemQuantity`, `Inventory`.`UnitsSold` AS `Inventory_UnitsSold`, `Inventory`.`Weight` AS `Inventory_Weight`, `Inventory`.`Size` AS `Inventory_Size`, `Inventory`.`Priority` AS `Inventory_Priority`, `Inventory`.`Location` AS `Inventory_Location`, `Inventory`.`Date` AS `Inventory_Date`, `Inventory`.`Dispose` AS `Inventory_Dispose` 
FROM `Inventory`
2025-08-17 19:06:09,698 INFO sqlalchemy.engine.Engine [cached since 2005s ago] {}
2025-08-17 19:06:09,699 INFO sqlalchemy.engine.Engine SELECT `Order`.`OrderId` AS `Order_OrderId`, `Order`.`ItemId` AS `Order_ItemId`, `Order`.`OrderQuantity` AS `Order_OrderQuantity`, `Order`.`Sales` AS `Order_Sales`, `Order`.`Price` AS `Order_Pric

In [92]:
inventory[0].ItemName

'Laptop'

In [93]:
def dbtoList(records):
    output_list = []
    for r in records:
        if isinstance(r, Inventory):
            data = {
                "ItemId": r.ItemId, 
                "ItemName": r.ItemName,
                "Category": r.ItemCategory,
                "Quantity": r.ItemQuantity, 
                "UnitsSold": r.UnitsSold,
                "Weight": r.Weight, 
                "Size": r.Size,
                "Priority": r.Priority, 
                "Location": r.Location,
                "Date": r.Date, 
                "Dispose": r.Dispose                
            }
        elif isinstance(r, Order):
            data = {
                "OrderId": r.OrderId,
                "ItemId": r.ItemId, 
                "OrderQuantity": r.OrderQuantity, 
                "Sales": r.Sales, 
                "Price": r.Price, 
                "Discount": r.Discount,
                "Profit": r.Profit, 
                "DateOrdered": r.DateOrdered,
                "DateReceived": r.DateReceived,
                "CustomerSegment": r.CustomerSegment
            }
        else:
            continue
        output_list.append(data)
    
    return output_list

inventoryData = dbtoList(inventory)
orderData = dbtoList(order)

## Supervised Models

In [ ]:
import pickle
import joblib
from datetime import datetime
from IPython.display import Markdown

class Supervised_Models:
    # Location Prediction Model
    def predict_location(input_data):
        with open('../../Supervised models/Samuel/storage_prediction_model.pkl', 'rb') as file:
            storage_prediction_model = pickle.load(file)
            
        categorical_features = {
            'Priority': ['High','Low','Medium'],
            'Product_Type': ['Clothing','Technology','Other','Sports and Fitness'],
            'Size': ['Large','Medium','Small']
        }
        numerical_features = ['Order_Quantity', 'Weight']
        one_hot_columns = []
        
        for feature, values in categorical_features.items():
            for value in values:
                one_hot_columns.append(f"{feature}_{value}")
            
        # Combine with numerical features to get all feature names
        all_feature_names = one_hot_columns + numerical_features

        features_dict = {col: 0 for col in all_feature_names}
    
        # Set one-hot encoded features
        for feature, values in categorical_features.items():
            if feature in input_data:
                selected_value = input_data[feature]
                one_hot_col = f"{feature}_{selected_value}"
                if one_hot_col in features_dict:
                    features_dict[one_hot_col] = 1
    
        # Set numerical features
        for feature in numerical_features:
            if feature in input_data:
                features_dict[feature] = float(input_data[feature])
        
        # Convert to array in the correct order
        features_array = np.array([features_dict[col] for col in all_feature_names]).reshape(1, -1)

        prediction = storage_prediction_model.predict(features_array)
        return prediction

    def demand_forecast_preprocessor(order_data, inventory_data):
        # Create Dataframe
        order = pd.DataFrame(order_data)
        inventory = pd.DataFrame(inventory_data)

        # Ensure dates are in datetime format
        order['DateOrdered'] = pd.to_datetime(order['DateOrdered'])
        
        # Extract Month (period or string, depending on preference)
        order['OrderMonth'] = order['DateOrdered'].dt.to_period('M')
        
        # Merge with Category lookup table
        merged_df = order.merge(inventory, on='ItemId', how='left')
        
        # Group and aggregate
        result_df = (
            merged_df
            .groupby(['OrderMonth', 'Category', 'CustomerSegment'], as_index=False)
            .agg(
                AveragePrice=('Price', 'mean'),
                AverageDiscount=('Discount', 'mean')
            )
        )
        
        return result_df

    # Demand forecast model
    def predict_demand_forecast(input_data):
        demand_forecast_model = joblib.load('../../Supervised models/ShernFai/model/salesforecast(categories).pkl')
        with open('../../Supervised models/ShernFai/model/salesforecast_preprocessor.pkl', 'rb') as f:
            preprocessor_data = pickle.load(f)

        categories = {
            "Clothing" : [
                "Cleats",
                "Men's Footwear",
                "Women's Apparel"
            ],
            "Technology": [
                "Electronics",
                "Video Games",
                "Cameras",
                "Computers",
            ],
            "Sports and Fitness": [
                "Cardio Equipment",
                "Indoor/Outdoor Games",
                "Water Sports",
                "Shop By Sport",
                "Camping & Hiking",
                "Fishing"
            ],
            "Other": [
                "Garden",
                "Pet Supplies"
            ]
        }

        cat_keys = list(categories.keys())

        # Extract preprocessor components
        le_category = preprocessor_data['label_encoder_category']
        reference_date = preprocessor_data['reference_date']
        unique_categories = preprocessor_data['unique_categories']
        feature_columns = preprocessor_data['feature_columns']

        # Get data
        category_name = input_data['category']
        future_month = input_data['month']
        avg_price = float(input_data['avg_price'])
        customer_segment = input_data['customer_segment']
        discount_rate = float(input_data['discount_rate'])
        
        # Parse the future date
        future_date = pd.to_datetime(future_month)
        
        # Calculate time features for the future date
        months_since_start = ((future_date - reference_date).days / 30.44)
        
        # Create test data with numerical time features
        test_data = {
            'Category Name': category_name,
            'Average Product Price': avg_price,
            'Customer Segment': customer_segment,
            'Order Item Discount Rate': discount_rate,
            # Time features (numerical - can handle ANY future date!)
            'Year': future_date.year,
            'Month': future_date.month,
            'Quarter': future_date.quarter,
            'Months_Since_Start': int(months_since_start),
            'Month_Sin': np.sin(2 * np.pi * future_date.month / 12),
            'Month_Cos': np.cos(2 * np.pi * future_date.month / 12),
            'Year_Trend': future_date.year - reference_date.year
        }
        
        # Create DataFrame
        test_df = pd.DataFrame([test_data])
        
        # Handle unknown category
        if category_name not in cat_keys:
            print(f"Unknown category '{category_name}' - using default: {cat_keys[0]}")
            test_df['Category Name'] = cat_keys[0]
            category_name = cat_keys[0]
        
        # One-hot encode customer segment
        test_df = pd.get_dummies(test_df, columns=['Customer Segment'], drop_first=True)
        
        # Ensure same columns as training (crucial!)
        test_df = test_df.reindex(columns=feature_columns, fill_value=0)
        
        # Make prediction
        total = 0
        num = len(categories[category_name])
        for subclass in categories[category_name]:
            test_df['Category Name'] = subclass
            test_df['Category Name'] = le_category.transform(test_df['Category Name'])
            total += demand_forecast_model.predict(test_df)
        
        avg_demand = total / num
        
        return avg_demand

    def detect_anomalies(inventory_list):
        anomalies_detected = []
        for item in inventory_list:
            current_location = item['Location']
            predicted_location = Supervised_Models.predict_location({
                "Priority": item['Priority'],
                "Product_Type": item['Category'],
                "Size": item['Size'],
                "Order_Quantity": item['Quantity'],
                "Weight": item['Weight']
            })[0]

            #print(f"\nCurrent Location: {current_location}")
            #print(f"Predicted Location: {predicted_location}")
            if current_location != predicted_location:
                #print(f"Anomaly detected! Item id:{item['ItemId']} is stored at location {current_location} while it should be stored at {predicted_location}.")
                anomalies_detected.append({'ItemId': item['ItemId'], 'ItemName': item['ItemName'], 'CurrentLocation': current_location, 'PredictedLocation': predicted_location})

        return anomalies_detected

s = Supervised_Models

In [119]:
print(inventoryData)

[{'ItemId': 1, 'ItemName': 'Laptop', 'Category': 'Technology', 'Quantity': 100, 'UnitsSold': 50, 'Weight': 1.5, 'Size': 10.0, 'Priority': 'High', 'Location': 'A-1', 'Date': '2025-06-01', 'Dispose': False}, {'ItemId': 2, 'ItemName': 'Chair', 'Category': 'Other', 'Quantity': 200, 'UnitsSold': 100, 'Weight': 2.0, 'Size': 20.0, 'Priority': 'Medium', 'Location': 'B-2', 'Date': '2025-07-01', 'Dispose': False}, {'ItemId': 3, 'ItemName': 'Laptop', 'Category': 'Technology', 'Quantity': 100, 'UnitsSold': 50, 'Weight': 1.5, 'Size': 10.0, 'Priority': 'High', 'Location': 'A-1', 'Date': '2025-06-01', 'Dispose': False}, {'ItemId': 4, 'ItemName': 'Chair', 'Category': 'Other', 'Quantity': 200, 'UnitsSold': 100, 'Weight': 2.0, 'Size': 20.0, 'Priority': 'Medium', 'Location': 'B-2', 'Date': '2025-07-01', 'Dispose': False}, {'ItemId': 5, 'ItemName': 'Laptop', 'Category': 'Technology', 'Quantity': 100, 'UnitsSold': 50, 'Weight': 1.5, 'Size': 10.0, 'Priority': 'High', 'Location': 'A-1', 'Date': '2025-06-01',

In [121]:
pd.DataFrame(inventoryData)

,ItemId,ItemName,Category,Quantity,UnitsSold,Weight,Size,Priority,Location,Date,Dispose
0,1,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
1,2,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
2,3,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
3,4,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
4,5,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
5,6,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
6,7,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
7,8,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
8,9,None,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
9,10,None,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False


In [125]:
print(s.detect_anomalies(inventoryData))

Laptop
Chair
Laptop
Chair
Laptop
Chair
Laptop
Chair
None
None
Laptop
Chair
[{'ItemId': 1, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 2, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 3, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 4, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 5, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 6, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 7, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 8, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 9, 'ItemName': None, 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 10, 'ItemName': None, 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 11, 'ItemName'

In [99]:
# Testing
print(s.predict_location({
    "Priority": "Medium",
    "Product_Type": "Sports and Fitness",
    "Size": "Medium",
    "Order_Quantity": 12,
    "Weight": 10.78
}))

['B-3']


In [100]:
result = s.demand_forecast_preprocessor(orderData, inventoryData)
result

,OrderMonth,Category,CustomerSegment,AveragePrice,AverageDiscount
0,2025-06,Technology,Corporate,100.0,10.0
1,2025-07,Other,Consumer,100.0,5.0


In [101]:
print(s.predict_demand_forecast({
    'category': "Clothing",
    'month': "2025-05",
    'avg_price': 10.0,
    'customer_segment': "Consumer",
    'discount_rate': 0.12
}))

[186.10974]


D:\Yong Meng Lee\anaconda3\envs\it3100\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Sections

## Products Overview

In [ ]:
# Product Overview: This section summarizes the overall performance of products
product_overview = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate an overview of products based on the following:

1. **Top Products**: Identify the top-selling products by sales volume and revenue over the past month.
2. **Product Performance**: Analyze the performance of each product category in terms of sales, demand, and revenue.
3. **Sales by Product**: Present a summary of sales for each product, including revenue, volume sold, and average price.

Data:
- Sales Data: {sales_data}
- Product Categories: {product_categories_data}
- Product Sales Volume: {sales_volume_data}
- Product Revenue: {product_revenue_data}
'''
)

display(Markdown(product_overview.text))


## Category Distribution

In [ ]:
# Category Distribution: This section analyzes the distribution of sales across categories
category_distribution = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate insights on the distribution of sales across categories based on the following:

1. **Category-wise Sales Volume**: Summarize the total sales volume across all product categories.
2. **Category-wise Revenue**: Display the total revenue generated from each product category over the past month.
3. **Category-wise Performance**: Provide insights on which product categories are performing the best in terms of sales, revenue, and customer demand.

Data:
- Sales Volume by Category: {sales_volume_by_category_data}
- Revenue by Category: {revenue_by_category_data}
- Product Category Performance: {category_performance_data}
'''
)

display(Markdown(category_distribution.text))


## Product Usage Forecast

In [ ]:
# Product Usage Forecast: This section forecasts the future usage of products based on historical sales
product_usage_forecast = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate a product usage forecast for the upcoming period based on historical data and trends:

1. **Sales Forecast by Product**: Predict the sales volume for each product in the next quarter/year.
2. **Demand Forecast**: Provide a demand forecast for products based on historical sales, usage probabilities, and seasonal patterns.
3. **Future Stock Levels**: Estimate the required stock levels for each product to meet forecasted demand.

Data:
- Historical Sales Data: {historical_sales_data}
- Product Usage Data: {usage_probabilities}
- Seasonal Patterns: {seasonal_sales_patterns_data}
- Current Stock Levels: {current_inventory_data}
'''
)

display(Markdown(product_usage_forecast.text))


## Sales Insights

In [ ]:
sales_data = orderData
sales_predictions = []
product_categories = ["Clothing","Technology","Sports and Fitness","Other"]
current_inventory = inventoryData
usage_probabilities = ""

section_sales_insights = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate a sales insights report that describes information for the following areas:

1. Sales Trends: Summarize sales based on the provided data. Provide insights on which product categories are seeing the highest demand.
2. Product Performance: Analyze the best-selling products by quantity. Highlight the top 3 performing products.
3. Product Demand Forecast: Based on the historical sales and usage probability, forecast the demand for the next month.
4. Restocking or Discontinuation: Recommend which products should be restocked and which should be discontinued, based on sales trends and inventory levels.

Data:
- Historical sales data: {sales_data}
- Sales volume predictions for next month: {sales_predictions}
- Product categories: {product_categories}
- Current inventory levels: {current_inventory}
- Usage probabilities: {usage_probabilities}'''
)

display(Markdown(section_sales_insights.text))

## Storage Optimizations

In [137]:
location_predictions = []
for item in inventoryData:
    location_predictions.append({
        'Item Id': item['ItemId'],
        'Current Location': item['Location'],
        'Predicted Location': s.predict_location({
            'Priority': item['Priority'],
            'Product_Type': item['Category'],
            'Size': item['Size'],
            'Order_Quantity': item['Quantity'],
            'Weight': item['Weight']
        })[0]
    })

section_storage_optimizations = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Provide detailed storage optimization recommendations based on:

1. Current storage utilization metrics
2. Model-predicted optimal locations vs current locations
3. List of items flagged for relocation, including:
   - Current location
   - Recommended location

Data:
{inventoryData}
{location_predictions}
'''
)

display(Markdown(section_storage_optimizations.text))

Based on the provided data, here are detailed storage optimization recommendations:

---

## Storage Optimization Recommendations Report

**Date:** 2023-10-27

**Objective:** To provide comprehensive recommendations for optimizing storage utilization, improving efficiency, and aligning physical inventory with model-predicted optimal locations.

---

### 1. Current Storage Utilization Metrics

Currently, inventory is distributed across two primary locations: `A-1` and `B-2`.

| Metric                   | Location A-1       | Location B-2       | Total (Combined)   |
| :----------------------- | :----------------- | :----------------- | :----------------- |
| **Primary Items Stored** | Laptop (High Pri.) | Chair (Medium Pri.)| Laptop, Chair, N/A |
| **Number of Unique Item SKUs** | 6                  | 6                  | 12                 |
| **Total Quantity of Items** | 600 units (6 SKUs x 100 units/SKU) | 1200 units (6 SKUs x 200 units/SKU) | 1800 units         |
| **Total Estimated Volume** | 6,000 cubic units (600 units x 10.0 size/unit) | 24,000 cubic units (1200 units x 20.0 size/unit) | 30,000 cubic units |
| **Total Estimated Weight** | 900 weight units (600 units x 1.5 weight/unit) | 2,400 weight units (1200 units x 2.0 weight/unit) | 3,300 weight units |
| **Total Units Sold (Velocity)** | 300 units (6 SKUs x 50 units sold/SKU) | 600 units (6 SKUs x 100 units sold/SKU) | 900 units          |
| **Average Priority**     | High               | Medium             | -                  |

**Observations:**

*   **Segregated Storage:** High-priority, smaller-volume items (Laptops) are in A-1, while medium-priority, larger-volume items (Chairs) are in B-2.
*   **Volume Imbalance:** Location B-2 currently houses significantly more volume and weight compared to A-1.
*   **Data Anomaly:** Two items (ItemId 9 & 10) have `ItemName: None`, indicating a potential data quality issue requiring investigation.

---

### 2. Model-Predicted Optimal Locations vs. Current Locations

The model uniformly recommends consolidating *all* inventory from both Location A-1 and Location B-2 into a single location: **B-5**.

| Item Type | Current Location | Model-Predicted Location | Number of Items Affected | Total Qty Affected | Total Volume Affected | Total Weight Affected |
| :-------- | :--------------- | :----------------------- | :----------------------- | :----------------- | :-------------------- | :-------------------- |
| Laptop    | A-1              | B-5                      | 6                        | 600                | 6,000 cubic units     | 900 weight units      |
| Chair     | B-2              | B-5                      | 6                        | 1,200              | 24,000 cubic units    | 2,400 weight units    |
| **Overall** | **A-1, B-2**     | **B-5**                  | **12**                   | **1,800**          | **30,000 cubic units**| **3,300 weight units**|

**Model Rationale (Inferred):**

The strong recommendation for consolidation into `B-5` suggests the model identifies significant benefits from a centralized storage approach. Possible reasons include:

*   **Improved Efficiency:** Reduced travel paths for picking and replenishment operations, especially if B-5 is a strategically located central hub.
*   **Optimized Space Utilization:** B-5 might be designed for higher density storage, or the consolidation allows for more flexible slotting of diverse items, maximizing cubic space.
*   **Simplified Inventory Management:** A single location simplifies inventory counts, stock rotation, and overall warehouse management.
*   **Cost Reduction:** Potential savings on labor, equipment utilization, and potentially, the ability to repurpose or de-commission the emptied locations (A-1, B-2).

---

### 3. List of Items Flagged for Relocation

All 12 unique Item IDs are flagged for relocation to B-5.

| Item ID | Item Name | Category   | Current Location | Recommended Location | Quantity | Units Sold | Weight (per unit) | Size (per unit) | Priority |
| :------ | :-------- | :--------- | :--------------- | :------------------- | :------- | :--------- | :---------------- | :-------------- | :------- |
| 1       | Laptop    | Technology | A-1              | B-5                  | 100      | 50         | 1.5               | 10.0            | High     |
| 3       | Laptop    | Technology | A-1              | B-5                  | 100      | 50         | 1.5               | 10.0            | High     |
| 5       | Laptop    | Technology | A-1              | B-5                  | 100      | 50         | 1.5               | 10.0            | High     |
| 7       | Laptop    | Technology | A-1              | B-5                  | 100      | 50         | 1.5               | 10.0            | High     |
| 9       | *None*    | Technology | A-1              | B-5                  | 100      | 50         | 1.5               | 10.0            | High     |
| 11      | Laptop    | Technology | A-1              | B-5                  | 100      | 50         | 1.5               | 10.0            | High     |
| 2       | Chair     | Other      | B-2              | B-5                  | 200      | 100        | 2.0               | 20.0            | Medium   |
| 4       | Chair     | Other      | B-2              | B-5                  | 200      | 100        | 2.0               | 20.0            | Medium   |
| 6       | Chair     | Other      | B-2              | B-5                  | 200      | 100        | 2.0               | 20.0            | Medium   |
| 8       | Chair     | Other      | B-2              | B-5                  | 200      | 100        | 2.0               | 20.0            | Medium   |
| 10      | *None*    | Other      | B-2              | B-5                  | 200      | 100        | 2.0               | 20.0            | Medium   |
| 12      | Chair     | Other      | B-2              | B-5                  | 200      | 100        | 2.0               | 20.0            | Medium   |

---

### 4. Detailed Storage Optimization Recommendations

Based on the analysis, the core recommendation is to proceed with the consolidation strategy suggested by the model, but with critical preceding validations and a detailed implementation plan.

#### 4.1. Pre-Relocation Validation (Critical)

1.  **Validate B-5 Capacity and Suitability:**
    *   **Physical Space:** Confirm B-5 has sufficient total cubic capacity (30,000 cubic units) to house all items.
    *   **Weight Bearing Capacity:** Ensure B-5's flooring, racking, and infrastructure can safely support the combined weight of 3,300 weight units.
    *   **Environmental Controls:** Verify B-5 meets any specific environmental requirements for Laptops (e.g., temperature, humidity).
    *   **Accessibility:** Assess access points, loading docks, and internal pathways for efficient movement of both small/light and large/heavy items.
    *   **Security:** Ensure B-5 has appropriate security measures for high-value items like Laptops.

2.  **Address Data Quality Issues:**
    *   Investigate `ItemId 9` and `ItemId 10` where `ItemName` is `None`. Accurately identify these items to ensure correct handling, slotting, and inventory tracking post-relocation.

#### 4.2. Strategic Relocation Planning

1.  **Phased Relocation Approach:**
    *   Given the significant volume, consider a phased move to minimize disruption to operations. Prioritize items based on ease of movement, or begin with lower-velocity items first.
    *   Ensure adequate labor and equipment (forklifts, pallet jacks, etc.) are available for the move.

2.  **Optimize B-5 Layout (Post-Relocation Principles):**
    *   **ABC Slotting:** Place high-velocity (Laptops - 50 units sold/SKU) and high-priority items in the most accessible and prime picking locations within B-5. Medium-velocity (Chairs - 100 units sold/SKU) can be placed in less prime but still accessible spots. *Note: While Chairs have higher units sold, Laptops have 'High' priority, suggesting their importance might override pure velocity for prime placement.*
    *   **Product Grouping (Family Grouping):** Keep similar items together (e.g., all Laptops in one section, all Chairs in another) to streamline picking and replenishment, even if they have different Item IDs.
    *   **Ergonomics and Safety:** Store heavier items (Chairs) at lower levels to reduce injury risk and facilitate handling.
    *   **Slot-to-Size Matching:** Utilize various racking and shelving configurations within B-5 to best fit the diverse sizes of Laptops (smaller) and Chairs (larger), maximizing cubic utilization.

3.  **Inventory Accuracy During Relocation:**
    *   Implement a rigorous tracking process during the move to ensure 100% inventory accuracy. This may involve scanning items out of current locations and into B-5.
    *   Consider a full inventory count immediately after the relocation to establish a clean baseline.

#### 4.3. Post-Relocation Actions

1.  **Review and Repurpose Locations A-1 & B-2:**
    *   Once fully vacated, assess the future of A-1 and B-2. Can they be:
        *   Used for future expansion?
        *   Repurposed for other business functions (e.g., staging, returns processing, kitting)?
        *   Leased out to generate revenue?
        *   Decommissioned if no longer needed?
    *   Estimate the cost savings associated with no longer maintaining these locations (if applicable).

2.  **Establish Performance Monitoring:**
    *   Track key performance indicators (KPIs) in B-5 post-relocation:
        *   Order pick time and accuracy.
        *   Space utilization percentage.
        *   Inventory turns.
        *   Damage rates.
        *   Labor efficiency.
    *   This will help validate the benefits of consolidation and identify areas for continuous improvement.

3.  **Regular Model Recalibration:**
    *   Inventory needs are dynamic. Periodically re-run the optimization model (e.g., quarterly or semi-annually) with updated sales data, new product introductions, and inventory levels to ensure the storage strategy remains optimal.

---

### Conclusion

The model's recommendation to consolidate all inventory into B-5 presents a significant opportunity for increased operational efficiency and potentially reduced costs. However, successful implementation hinges on thorough validation of B-5's capabilities and a meticulously planned relocation process. By addressing data quality, optimizing the internal layout of B-5, and establishing ongoing performance monitoring, this optimization initiative can yield substantial benefits for the organization's storage and logistics operations.

## Anomalies Detected

In [141]:
section_anomalies_detected = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate an anomalies section that lists all detected storage anomalies detected in a table. Include each item's current location, predicted location, item id, and name.
Include the reason for each anomaly.

Data:
{s.detect_anomalies(inventoryData)}
'''
)

display(Markdown(section_anomalies_detected.text))

## Detected Storage Anomalies

The following anomalies have been detected in the storage system, indicating discrepancies between actual and predicted item states, or missing critical information.

---

**Anomaly 1: Location Discrepancy**
*Reason:* The item's current physical location does not match its predicted storage location. This indicates either a misplacement, an unrecorded movement, or an outdated prediction.

- **Item ID:** 1
  **Item Name:** Laptop
  **Current Location:** A-1
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 2
  **Item Name:** Chair
  **Current Location:** B-2
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 3
  **Item Name:** Laptop
  **Current Location:** A-1
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 4
  **Item Name:** Chair
  **Current Location:** B-2
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 5
  **Item Name:** Laptop
  **Current Location:** A-1
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 6
  **Item Name:** Chair
  **Current Location:** B-2
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 7
  **Item Name:** Laptop
  **Current Location:** A-1
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 8
  **Item Name:** Chair
  **Current Location:** B-2
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 11
  **Item Name:** Laptop
  **Current Location:** A-1
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

- **Item ID:** 12
  **Item Name:** Chair
  **Current Location:** B-2
  **Predicted Location:** B-5
  **Reason for Anomaly:** Location Discrepancy

---

**Anomaly 2: Missing Item Information**
*Reason:* The item's name is missing or undefined, hindering proper identification, tracking, and management.

- **Item ID:** 9
  **Item Name:** N/A (Missing)
  **Current Location:** A-1
  **Predicted Location:** B-5
  **Reason for Anomaly:**
    *   Location Discrepancy: The item's current physical location (A-1) does not match its predicted storage location (B-5).
    *   Missing Item Information: The item's name is undefined.

- **Item ID:** 10
  **Item Name:** N/A (Missing)
  **Current Location:** B-2
  **Predicted Location:** B-5
  **Reason for Anomaly:**
    *   Location Discrepancy: The item's current physical location (B-2) does not match its predicted storage location (B-5).
    *   Missing Item Information: The item's name is undefined.

## Summary

# PDF File Generation

In [220]:
from markdown_pdf import MarkdownPdf, Section

# Read Markdown content
mdc = f'''<h1 style="text-align:center;">Monthly Report</h1><br>

# Products Overview:
section_products_overview.text

# Category Distribution:
section_category_distribution.text

# Product Usage Forecast:
section_product_usage.text

# Sales Insights:
section_sales_insights.text

# Storage Optimizations:
{section_storage_optimizations.text}

# Anomalies Detected:
{section_anomalies_detected.text}

# Summary:
section_summary.text
'''
pdf = MarkdownPdf()
pdf.add_section(Section(mdc)) # Add Section(md_content, user_css=css_content) for custom CSS
pdf.save("MonthlyReport.pdf")